# Day 7

Full power to thrusters! We need to send the correct inputs to the correct amplifiers in order to get the maximum possible thruster output. The codes for the thruster will run on our existing intcode computer :grimace:


# Part 1

The elves don't remember which inputs go to which thrusters. So, given the thruster program, assign the inputs to the amplimfiers in such a way to produce the maximum thruster signal output

In [1]:
import math

from itertools import permutations
from typing import Tuple, List, Union

In [9]:
class Computer:
    def __init__(self, memory: List[int], pointer: int = 0, inputs: List[int] = None):
        self.memory = memory.copy()
        self.pointer = pointer
        self.inputs = inputs if inputs else []
        self.outputs = []
        self.complete = False
        self.pointer_increment_dict = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4}
    
    @staticmethod
    def parse_opcode(opcode: int) -> Tuple[int]:
        opcode_param_list = [int(num) for num in str(opcode).zfill(5)]
        # Don't need the leading zero from the opcode
        return tuple(opcode_param_list[:3] + opcode_param_list[4:])
    
    def get_output_positions(self, first_param_mode: int, second_param_mode: int) -> Tuple[int]:
        first_output_position = self.pointer + 1 if first_param_mode == 1 else self.memory[self.pointer + 1]
    
        try:
            second_output_position = self.pointer + 2 if second_param_mode == 1 else self.memory[self.pointer + 2]
        except IndexError:
            return (first_output_position, 0)

        return (first_output_position, second_output_position)
    
    def run_instruction(self, opcode: int, debug: bool = False):
        # Third param mode is always in position mode
        _, second_param_mode, first_param_mode, opcode = self.parse_opcode(opcode)
        first_output_position, second_output_position = self.get_output_positions(first_param_mode, second_param_mode)
        
        try:
            replace_idx = self.memory[self.pointer + 3]
        except IndexError:
            pass

        if opcode == 4:
            if debug:
                print(f"Adding ouput: {self.memory[first_output_position]} to outputs: {self.outputs}")
            self.outputs += [self.memory[first_output_position]]
        elif opcode == 5 and self.memory[first_output_position] != 0:
            if debug:
                print(f"Setting address from {self.pointer} to {self.memory[second_output_position]}")
            self.pointer = self.memory[second_output_position]
            # Early return so we don't change the pointer at the end of the fx
            return
        elif opcode == 6 and self.memory[first_output_position] == 0:
            if debug:
                print(f"Setting address from {self.pointer} to {self.memory[second_output_position]}")
            self.pointer = self.memory[second_output_position]
            # Early return so we don't change the pointer at the end of the fx
            return

        if opcode == 1:
            if debug:
                print(f"Replacing {self.memory[replace_idx]} at address {replace_idx} with {self.memory[first_output_position] + self.memory[second_output_position]}")
            self.memory[replace_idx] = self.memory[first_output_position] + self.memory[second_output_position]
        elif opcode == 2:
            if debug:
                print(f"Replacing {self.memory[replace_idx]} at address {replace_idx} with {self.memory[first_output_position] * self.memory[second_output_position]}")
            self.memory[replace_idx] = self.memory[first_output_position] * self.memory[second_output_position]
        elif opcode == 3:
            if debug:
                print(f"Using next input from: {self.inputs}")
            computer_input = self.inputs.pop(0)
            if debug:
                print(f"Replacing {self.memory[first_output_position]} at {first_output_position} with {computer_input}")
            self.memory[first_output_position] = computer_input
        elif opcode == 7:
            replacement = 1 if self.memory[first_output_position] < self.memory[second_output_position] else 0
            if debug:
                print(f"Replacing {self.memory[replace_idx]} at address {replace_idx} with {replacement}")
            self.memory[replace_idx] = replacement
        elif opcode == 8:
            replacement = 1 if self.memory[first_output_position] == self.memory[second_output_position] else 0
            if debug:
                print(f"Replacing {self.memory[replace_idx]} at address {replace_idx} with {replacement}")
            self.memory[replace_idx] = replacement
        
        if debug:
            print(f"Setting address from {self.pointer} to {self.pointer + self.pointer_increment_dict.get(opcode)}")
        # Increment pointer
        self.pointer += self.pointer_increment_dict.get(opcode)
    
    def run_program(self, debug=False):
        while not self.complete:
            opcode = self.memory[self.pointer]
            if debug:
                print(f"opcode: {opcode} at address: {self.pointer}")
                print(f"memory before instruction: {self.memory}")
            
            if opcode == 99:
                self.complete = True
            elif str(opcode).endswith("3") and not self.inputs:
                if debug:
                    print(f"No more inputs, exiting")
                break
            else:
                self.run_instruction(opcode, debug)
            
            if debug:
                print(f"memory after instruction: {self.memory}")
            
            
        

In [10]:
# Make sure that intcode computer refactoring works with this
# Had to refactor to account for multiple 3 opcodes within an intcode program
# Also had to refactor to account for 4 opcode is allowed to spit out non zero output before end of "test"
# I think this has to do with the fact that we're not running tests anymore
program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
computer = Computer(program, inputs=[4, 0]) 
computer.run_program()
assert computer.outputs == [4]
computer = Computer(program, inputs=[3, 4]) 
computer.run_program()
assert computer.outputs == [43]
computer = Computer(program, inputs=[2, 43]) 
computer.run_program()
assert computer.outputs == [432]
computer = Computer(program, inputs=[1, 432]) 
computer.run_program()
assert computer.outputs == [4321]
computer = Computer(program, inputs=[0, 4321]) 
computer.run_program()
assert computer.outputs == [43210]

In [36]:
class Amp:
    def __init__(self, program: List[int], setting: int, input_signal: int = 0):
        self.computer = Computer(program)
        self.setting = setting
        # Add setting when we start
        self.computer.inputs += [setting]
        self.input_signal = input_signal
    
    def produce_output_signal(self, debug: bool =  False) -> int:
        if debug:
            print(f"---Running amp with setting: {self.setting} and input signal: {self.input_signal}")
        self.computer.inputs += [self.input_signal]
        self.computer.run_program(debug)
        output_signal = self.computer.outputs[-1]
        if debug:
            print(f"---Producing output signal {output_signal}")
        return output_signal

In [37]:
program_1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
amp_a1 = Amp(program_1, setting=4)
assert amp_a1.produce_output_signal() == 4
amp_b1 = Amp(program_1, setting=3, input_signal=4)
assert amp_b1.produce_output_signal() == 43
program_2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
amp_a2 = Amp(program_2, setting=0)
assert amp_a2.produce_output_signal() == 5
amp_b2 = Amp(program_2, setting=1, input_signal=5)
assert amp_b2.produce_output_signal() == 54

In [38]:
class Thruster:
    def __init__(self, program: List[int], amp_settings: tuple):
        self.amps = [Amp(program, setting) for setting in amp_settings]
    
    def produce_direct_output_signal(self, initial_signal: int = 0, debug: bool = False) -> int:
        output_signal = initial_signal
        for amp in self.amps:
            amp.input_signal = output_signal
            if debug:
                print("*****Begin Amp Run*****")
            output_signal = amp.produce_output_signal(debug)
            if debug:
                print("*****End Amp Run*****")
            # Adding this gives me the right answer for the tests, but then makes the actual input run forever
            # Not adding produces answers that are too small for both the tests and the actual input
            #amp.computer.pointer = 0
        return output_signal
    
    def produce_looped_output_signal(self, initial_signal: int = 0, debug: bool = False) -> int:
        output_signal = initial_signal
        while not self.amps[-1].computer.complete:
            output_signal = self.produce_direct_output_signal(output_signal, debug)
        return output_signal
            

In [39]:
program_1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
program_1_settings = (4,3,2,1,0)
thruster_1 = Thruster(program_1, program_1_settings)
assert thruster_1.produce_direct_output_signal() == 43210
program_2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
program_2_settings = (0,1,2,3,4)
thruster_2 = Thruster(program_2, program_2_settings)
assert thruster_2.produce_direct_output_signal() == 54321
program_3 = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
program_3_settings = (1,0,4,3,2)
thruster_3 = Thruster(program_3, program_3_settings)
assert thruster_3.produce_direct_output_signal() == 65210

In [40]:
all_possible_settings = permutations((0, 1, 2, 3, 4))
thruster_program = [3,8,1001,8,10,8,105,1,0,0,21,46,59,84,93,110,191,272,353,434,99999,3,9,101,2,9,9,102,3,9,9,1001,9,5,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,102,5,9,9,4,9,99,3,9,1001,9,4,9,1002,9,2,9,101,2,9,9,102,2,9,9,1001,9,3,9,4,9,99,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,1001,9,5,9,1002,9,3,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,99]
max_output_signal = -float("inf")

for amp_settings in all_possible_settings:
    thruster = Thruster(thruster_program, amp_settings)
    output_signal = thruster.produce_direct_output_signal()
    max_output_signal = output_signal if output_signal > max_output_signal else max_output_signal
    
print(max_output_signal)

19650


# Part 2

It's not enough juice to get the thrusters going :cry: But we can't give up, Santa is counting on us! The elves talked me through rewiring the amplifiers into a feedback loop -- now we can start cooking!

In [42]:
thruster_program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
amplifier_settings =(9, 8, 7, 6, 5)
thruster = Thruster(thruster_program, amplifier_settings)
assert thruster.produce_looped_output_signal() == 139629729

thruster_program = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
amplifier_settings = (9, 7, 8, 5, 6)
thruster = Thruster(thruster_program, amplifier_settings)
assert thruster.produce_looped_output_signal() == 18216

In [44]:
all_possible_settings = permutations((5, 6, 7, 8, 9))
thruster_program = [3,8,1001,8,10,8,105,1,0,0,21,46,59,84,93,110,191,272,353,434,99999,3,9,101,2,9,9,102,3,9,9,1001,9,5,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,102,5,9,9,4,9,99,3,9,1001,9,4,9,1002,9,2,9,101,2,9,9,102,2,9,9,1001,9,3,9,4,9,99,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,1001,9,5,9,1002,9,3,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,99]
max_output_signal = -float("inf")

for amplifier_settings in all_possible_settings:
    thruster = Thruster(thruster_program, amplifier_settings)
    output_signal = thruster.produce_looped_output_signal()
    max_output_signal = output_signal if output_signal > max_output_signal else max_output_signal
    
print(max_output_signal)

35961106
